# Elecciones

## Datasets

### Características Socioeconómicas y demográficas

In [35]:
from tabulate import tabulate
import pandas as pd

# Cargar datos
all_data = pd.read_csv("data/county_complete.csv", header=0, sep=",")

# Seleccionar las primeras tres columnas y las columnas que contienen "2019"
all_data = all_data.iloc[:, [0, 1, 2]].join(all_data.loc[:, all_data.columns.str.contains("2019")])

# Renombrar la tercera columna
all_data.columns.values[2] = "county"

# Usar tabulate para mostrar las primeras 5 filas en formato de tabla
print(tabulate(all_data.head(), headers='keys', tablefmt='fancy_grid'))
print(len(all_data))

╒════╤════════╤═════════╤════════════════╤════════════════════╤════════════════════╤════════════════════╤════════════════════╤══════════════╤════════════════════════╤══════════════════╤══════════════╤═════════════════╤════════════════════════════════╤═══════════════════════════════╤═════════════════════════════════╤═══════════════════╤══════════════════════════════════════════╤═════════════════════════════════════════╤═══════════════════════════════╤══════════════════════════════════════════════╤═════════════════════════════════╤═════════════════════════════╤════════════════════════════════════╤════════════════════════════════════╤════════════════╤══════════════════════════════╤═════════════════════════╤═══════════════════╤════════════════════════════════╤═════════════════════════════════╤════════════════════════════════════════════╤═══════════════╤══════════════════════════╤════════════════╤══════════════════════════╤══════════════════════════════╤════════════╤════════════════╤═══════

### Resultados electorales

In [36]:
# Cargar datos
d1 = pd.read_csv("data/2020_US_County_Level_Presidential_Results.csv")

# Crear nueva columna 'votes_others'
d1['votes_others'] = d1['total_votes'] - (d1['votes_gop'] + d1['votes_dem'])

# Quedarse con columnas específicas
d1 = d1[['state_name', 'county_fips', 'county_name', 'votes_gop', 'votes_dem', 'total_votes', 'votes_others']]

# Renombrar columnas
d1.columns = ["state", "fips", "county", "Trump_2020", "Biden_2020", "Total_2020", "Other_2020"]

# Usar tabulate para mostrar las primeras 5 filas en formato de tabla
print(tabulate(d1.head(), headers='keys', tablefmt='fancy_grid'))
print(len(d1))

╒════╤═════════╤════════╤════════════════╤══════════════╤══════════════╤══════════════╤══════════════╕
│    │ state   │   fips │ county         │   Trump_2020 │   Biden_2020 │   Total_2020 │   Other_2020 │
╞════╪═════════╪════════╪════════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│  0 │ Alabama │   1001 │ Autauga County │        19838 │         7503 │        27770 │          429 │
├────┼─────────┼────────┼────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│  1 │ Alabama │   1003 │ Baldwin County │        83544 │        24578 │       109679 │         1557 │
├────┼─────────┼────────┼────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│  2 │ Alabama │   1005 │ Barbour County │         5622 │         4816 │        10518 │           80 │
├────┼─────────┼────────┼────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│  3 │ Alabama │   1007 │ Bibb County    │         7525 │         1986 │ 

### Dataset final

In [39]:
# Unir los dos datasets
data = pd.merge(d1, all_data, how='inner', on=["county", "state"])

# Usar tabulate para mostrar las primeras 5 filas en formato de tabla
print(tabulate(data.head(), headers='keys', tablefmt='fancy_grid'))
print(len(data))

╒════╤═════════╤══════════╤════════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════╤════════════════════╤════════════════════╤════════════════════╤════════════════════╤══════════════╤════════════════════════╤══════════════════╤══════════════╤═════════════════╤════════════════════════════════╤═══════════════════════════════╤═════════════════════════════════╤═══════════════════╤══════════════════════════════════════════╤═════════════════════════════════════════╤═══════════════════════════════╤══════════════════════════════════════════════╤═════════════════════════════════╤═════════════════════════════╤════════════════════════════════════╤════════════════════════════════════╤════════════════╤══════════════════════════════╤═════════════════════════╤═══════════════════╤════════════════════════════════╤═════════════════════════════════╤════════════════════════════════════════════╤═══════════════╤══════════════════════════╤════════════════╤══════════════════════